# Find Data Issues


## First, load and process all the data

In [5]:
def date(x):
    if type(x) == float:
        return 'null'
    else:
        return(pd.to_datetime(list(x.values())[0], unit='ms'))

def date1(x):
    if type(x) == float:
        return 'null'
    else:
        return(x)

def explode(df, col):
    df[col] = df[col].apply(lambda x: [x] if not isinstance(x, list) else x)
    return df.drop(col, axis=1).join(pd.DataFrame(list(df[col])).stack().reset_index(level=1, drop=True).rename(col)) 

def explode1(x, key):
    if type(x) == dict and key in x.keys(): return x[key]

def checkKey(dic, key):
    if key in dic.keys():
        return True
    else:
        return False

import json    
import pandas as pd
import datetime

receipt_data = pd.read_json("receipts.json", lines=True)
receipt_data['_id'] = receipt_data['_id'].apply(lambda x: list(x.values())[0])
receipt_data['createDate'] = receipt_data['createDate'].apply(lambda x: date(x))
receipt_data['dateScanned'] = receipt_data['dateScanned'].apply(lambda x: date(x))
receipt_data['finishedDate'] = receipt_data['finishedDate'].apply(lambda x: date(x))
receipt_data['modifyDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))
receipt_data['pointsAwardedDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))
receipt_data['purchaseDate'] = receipt_data['finishedDate'].apply(lambda x: date1(x))

receipt_data = receipt_data.rename(columns={'_id':'Id'})
receipt_tb = pd.DataFrame(receipt_data, columns=["Id", "bonusPointsEarned", "bonusPointsEarnedReason", "createDate", "dateScanned", "finishedDate", "modifyDate", "pointsAwardedDate", "pointsEarned", "purchaseDate", "purchasedItemCount", "rewardsReceiptStatus", "totalSpent", "userId"])

receipt_item_data = receipt_data[['rewardsReceiptItemList','Id']].copy()
receipt_item_data = explode(receipt_item_data,"rewardsReceiptItemList")

keys_ = ['barcode', 'description', 'finalPrice', 'itemPrice', 'needsFetchReview', 'partnerItemId', 'quanityPurchased']

receipt_item_data1 = pd.DataFrame(columns=keys_)
for i in range(len(keys_)):
    this_column = receipt_item_data1.columns[i]
    receipt_item_data1[this_column] = receipt_item_data['rewardsReceiptItemList'].apply(lambda x: explode1(x,this_column))

receipt_item_data1 = pd.concat([receipt_item_data,receipt_item_data1],axis=1)
receipt_item_data1 = pd.DataFrame(receipt_item_data1, columns=keys_)

brand_data = pd.read_json("brands.json", lines=True)
brand_tb = pd.DataFrame(brand_data, columns=["barcode", "category", "categoryCode", "name", "topBrand", "brandCode"])

user_data = pd.read_json("users.json", lines=True)
user_data['_id'] = user_data['_id'].apply(lambda x: list(x.values())[0])
user_data = user_data.rename(columns={'_id':'Id'})
user_data['createdDate'] = user_data['createdDate'].apply(lambda x: date(x))
user_data['lastLogin'] = user_data['lastLogin'].apply(lambda x: date(x))
user_tb = pd.DataFrame(user_data, columns=["_id", "state", "createdDate", "lastLogin", "role", "active", "signUpSource"])

## Then, find the data issues

### Issue 1: Data duplication
Below I will give detailed data for explanation

In [7]:
receipt_duplication_count = receipt_tb.duplicated().value_counts()
brand_duplication_count = brand_tb.duplicated().value_counts()
receipt_item_duplication_count = receipt_item_data1.duplicated().value_counts()
user_duplication_count = user_tb.duplicated().value_counts()

# print the duplication count info
print('Receipt duplication:', receipt_duplication_count, '\n')
print('Brand duplication:', brand_duplication_count, '\n')
print('Receipt Item duplication:', receipt_item_duplication_count, '\n')
print('User duplication:', user_duplication_count, '\n')

Receipt duplication: False    1119
Name: count, dtype: int64 

Brand duplication: False    1167
Name: count, dtype: int64 

Receipt Item duplication: False    5939
True     1442
Name: count, dtype: int64 

User duplication: True     283
False    212
Name: count, dtype: int64 



From the above data, we can see that there are 1442 and 283 duplicate records in the receipt item and user tables respectively, accounting for a large proportion of the total number of records.

### Issue 2: Data missing
Below I will give detailed data for explanation

In [8]:
receipt_missing_count = receipt_tb.isnull().sum()
brand_missing_count = brand_tb.isnull().sum()
receipt_item_missing_count = receipt_item_data1.isnull().sum()
user_missing_count = user_tb.isnull().sum()

In [9]:
print('Receipt null counts \n', receipt_missing_count)

Receipt null counts 
 Id                           0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate                 0
modifyDate                   0
pointsAwardedDate            0
pointsEarned               510
purchaseDate                 0
purchasedItemCount         484
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64


purchasedItemCount and totalSpent are very important information, too much missing data will make the data meaningless.

In [10]:
print('Brand null counts \n', brand_missing_count)

Brand null counts 
 barcode           0
category        155
categoryCode    650
name              0
topBrand        612
brandCode       234
dtype: int64


Generally speaking, both category and brandCode are attributes that cannot be null.

In [11]:
print('Receipt item null counts \n', receipt_item_missing_count)

Receipt item null counts 
 barcode             4291
description          821
finalPrice           614
itemPrice            614
needsFetchReview    6568
partnerItemId        440
quanityPurchased    7381
dtype: int64


Similarly, many necessary information of the item are confirmed.

In [12]:
print('User null counts \n', user_missing_count)

User null counts 
 _id             495
state            56
createdDate       0
lastLogin         0
role              0
active            0
signUpSource     48
dtype: int64


User's information is basically complete, and the missing information is basically very little.